In [1]:
# Daily net generation by balancing authority and energy source. Source: Form EIA-930 Product: Hourly Electric Grid Monitor. Data link: https://api.eia.gov/v2/electricity/rto/daily-fuel-type-data/data/?frequency=daily&data[0]=value&facets[respondent][]=US48&facets[timezone][]=Eastern&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000
# Daily demand and net generation by balancing authority. Source: Form EIA-930 Product: Hourly Electric Grid Monitor. Data link: https://api.eia.gov/v2/electricity/rto/daily-region-data/data/?frequency=daily&data[0]=value&facets[respondent][]=US48&facets[timezone][]=Eastern&facets[type][]=D&facets[type][]=NG&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000

#Power market/region mapping:
#Power markets               EIA code              type
#ERCOT ERCO     ISO
#NYISO NYIS       ISO
#CAISO CISO      ISO
#MISO  MISO     RTO
#PJM     PJM       RTO
#SPP      SWPP    RTO
#ISO-NE               ISNE       RTO
#Southwest       SW         power market
#Southeast         SE           power market
#Northwest       NW        power market
#US total              US48                      .
#Map    https://www.ferc.gov/electric-power-markets  

#Region               EIA code
#California          CAL
#Northwest       NW
#Southwest       SW
#Texas  TEX
#Central               CENT
#Midwest           MIDW
#Gulf     SE?
#TVA     TVA
#Southern          SE?
#Florida                FLO
#Mid-atlantic     MIDA
#Carolinas           CAR
#New York          NY
#New England   NE
#US total              US48                      .
#Map    https://www.eia.gov/todayinenergy/detail.php?id=4270

import pandas as pd
import requests # to request/download data via API
from datetime import datetime, timedelta # to break up start & end dates into smaller monthly intervals
from dateutil import rrule # to break up start & end dates into smaller monthly intervals
from dateutil.relativedelta import relativedelta # to break up start & end dates into smaller monthly intervals
from dotenv import load_dotenv # I need this to hide API key
import os # I need this to hide API key
import plotly.express as px # for charting the data

# I need this to hide API key
load_dotenv()
api_key = os.getenv('api_key')

# declare variables
start_date = datetime(2019, 11, 1) # start date for downloading data
end_date = datetime(2022, 1, 1) # end date for downloading data
startDates = [] # list to store start dates
endDates = [] # list to store end dates
list_data = [] # list to store data downloaded via API in json format

def API_function(start_date, end_date, list_data):
    # Download raw data using API
    URL = 'https://api.eia.gov/v2/electricity/rto/daily-fuel-type-data/data/?api_key=' + api_key + '&frequency=daily&data[0]=value&facets[respondent][]=US48&facets[timezone][]=Eastern&start=' + start_date.strftime('%Y-%m-%d') + '&end=' + end_date.strftime('%Y-%m-%d') + '&sort[0][column]=period&sort[0][direction]=desc&offset=0&out=json' # convert datetime format (e.g. '2019, 1, 1') to string format (e.g. '2019-01-01') for start and end dates so that these can be understood by this API request
    #URL = 'https://api.eia.gov/v2/electricity/rto/daily-region-data/data/?api_key=' + api_key + '&frequency=daily&data[0]=value&facets[respondent][]=US48&facets[timezone][]=Eastern&facets[type][]=D&facets[type][]=NG&sort[0][column]=period&start=' + start_date.strftime('%Y-%m-%d') + '&end=' + end_date.strftime('%Y-%m-%d') + '&sort[0][direction]=desc&offset=0&out=json' # convert datetime format (e.g. '2019, 1, 1') to string format (e.g. '2019-01-01') for start and end dates so that these can be understood by this API request
    #URL = 'https://api.eia.gov/series/?series_id=PET.MCRRIP12.M&api_key=' + api_key + '&out=json'
    #URL = 'http://api.eia.gov/category/?api_key=' + api_key + '&category_id=3&out=json'
    #URL = 'https://api.eia.gov/v2/electricity/retail-sales&api_key=' + api_key + '&out=json'
    headers = {'Accept-Encoding': 'identity'}
    r = requests.get(URL, headers=headers)
    json_data = r.json()

    # convert json data format to pandas DataFrame, then extract data as 'temp_list_data' list of dictionaries, then append to 'list_data' list of dictionaries and return it
    df_json = pd.DataFrame(json_data)
    temp_list_data = df_json.at['data', 'response']
    list_data.append(temp_list_data) 
    return list_data

# because each API request is limited to 5000 rows, I need to split the API request into several smaller chunks by creating a code to loop through start & end dates in 1-month increments.
for dt in rrule.rrule(rrule.MONTHLY, dtstart=start_date, until=end_date): # create 'startDates' list and 'endDates' list by looping through start & end dates in monthly increments
    startDates.append(dt) # add start-of-month date to 'startDates' list
    dt = dt + relativedelta(months=+1, days=-1) # calculate end of month date
    endDates.append(dt) # add end-of-month date to 'endDates' list
for start, end in zip(startDates, endDates): # runs 'API_function' function request for each pair of start and end dates stored in 'startDates' and 'endDates' lists and saves the result of each request it to 'list_data' list
    list_data = API_function(start, end, list_data)

# convert data from list of dictionaries format to pandas DataFrame format
col_names = list_data[0][0].keys() # create list of column names as 'col_names' dictionary
col_names = list(col_names) # convert 'col_names' dictionary to list
list_data = [item for sublist in list_data for item in sublist] # flatten 'list_data' list of lists of dictionaries into 'list_date' list of dictionaries in order to be able to parse it in creating a 'df' DataFrame below
df = pd.DataFrame(list_data, columns=col_names) # convert 'list_data' list of dictionaries to 'df' pandas DataFrame
df

# analysing daily powergen by fuel type to see when on Dec 15 & 24 there were power cuts that impacted southern and eastern US and specifically DUK, ETR, TVA as well as SRE, CNP, AGR, ES, SO, AEP
# In November, NERC warned that a large portion of North America was at risk of insufficient electricity supplies during peak winter conditions, due to higher demand projections, generator retirements, vulnerability to extreme weather and fuel supply and natural gas infrastructure limitations.

# group data by 'period' and 'type-name' coluumns, sum values in 'value' column, then convert period to datetime64 format, then sort by 'period' column
df = df.groupby(['period','type-name'])['value'].sum() # group by 'period' and 'type-name' coluumns and sum values in 'value' column
df = df.reset_index() # add row index column in order to move 'period' into a standalone column
#df.head(20)
#display(df.dtypes)
df['period'] = pd.to_datetime(df['period']) # change values in 'period' column from 'object' to 'datetime64'
#display(df.dtypes)
df.sort_values(by='period', inplace=True) # sort by date using 'period' column
#df

# calculate YoY diff and as % change 
df = df.pivot(index='period', columns='type-name', values='value') # move fuel types into columns. I need this to be able to calcuulate YoY change (actual & percentage) by fuel type
df = df.diff(periods=365).dropna() # calculate YoY diff (not percentage change but volume change), then delete rows with NaN values.
df_pct = df.pct_change(periods=365).dropna()*100 # calculate YoY % change, then delete rows with NaN values.
display(df) # show YoY diff
display(df_pct) # show YoY % change

# calculate 3-day rolling rolling average of YoY diff & save it in new DataFrame
df_mavg = df.rolling(3).mean().round(2).dropna() # 3-day rolling average
df_mavg.columns = [str(col) + '_3D_avg' for col in df_mavg.columns]
display(df_mavg)

# calculate 3-day rolling rolling average of YoY % change & save it in new DataFrame
df_mavg_pct = df_pct.rolling(3).mean().round(2).dropna() # 3-day rolling average
df_mavg_pct.columns = [str(col) + '_3D_avg' for col in df_mavg_pct.columns]
display(df_mavg_pct)

# add row index column in order to move 'period' into a standalone column
df.reset_index(inplace=True)
df_pct.reset_index(inplace=True)
df_mavg.reset_index(inplace=True)
df_mavg_pct.reset_index(inplace=True)

# unpivot DataFrames in order to chart it later
df = df.melt(id_vars=['period'], var_name='type-name', value_name='value')
df_pct = df_pct.melt(id_vars=['period'], var_name='type-name', value_name='value')
df_mavg = df_mavg.melt(id_vars=['period'], var_name='type-name', value_name='value')
df_mavg_pct = df_mavg_pct.melt(id_vars=['period'], var_name='type-name', value_name='value')

display(df)
display(df_pct)
display(df_mavg)
display(df_mavg_pct)

fig = px.line(df, x="period", y="value", color='type-name', title="US Lower 48: Net power generation (∆ MWh YoY)")
#fig.update_yaxes(fixedrange=True) # make only X axis scrollable
fig.show()

#df_nooil = df[df['type-name'] != 'Petroleum']
#fig = px.line(df_nooil, x="period", y="value", color='type-name', title="US Lower 48: Net power generation excluding petroleum (∆ MWh YoY)")
#fig.update_yaxes(fixedrange=True) # make only X axis scrollable
#fig.show()

fig = px.line(df_pct, x="period", y="value", color='type-name', title="US Lower 48: Net power generation (% chg YoY)")
#fig.update_yaxes(fixedrange=True) # make only X axis scrollable
fig.show()

fig = px.line(df_mavg, x="period", y="value", color='type-name', title="US Lower 48: Net power generation (∆ MWh YoY 3D_MAVG)")
#fig.update_yaxes(fixedrange=True) # make only X axis scrollable
fig.show()

fig = px.line(df_mavg_pct, x="period", y="value", color='type-name', title="US Lower 48: Net power generation (% chg YoY 3D_MAVG)")
#fig.update_yaxes(fixedrange=True) # make only X axis scrollable
fig.show()

,period,respondent,respondent-name,fueltype,type-name,timezone,timezone-description,value,value-units
0,2019-11-30,US48,United States Lower 48,NUC,Nuclear,Eastern,Eastern,2399578,megawatthours
1,2019-11-30,US48,United States Lower 48,SUN,Solar,Eastern,Eastern,94810,megawatthours
2,2019-11-30,US48,United States Lower 48,OTH,Other,Eastern,Eastern,196429,megawatthours
3,2019-11-30,US48,United States Lower 48,COL,Coal,Eastern,Eastern,1896420,megawatthours
4,2019-11-30,US48,United States Lower 48,WAT,Hydro,Eastern,Eastern,691028,megawatthours
...,...,...,...,...,...,...,...,...,...
6579,2022-01-01,US48,United States Lower 48,WND,Wind,Eastern,Eastern,1485777,megawatthours
6580,2022-01-01,US48,United States Lower 48,OTH,Other,Eastern,Eastern,184933,megawatthours
6581,2022-01-01,US48,United States Lower 48,SUN,Solar,Eastern,Eastern,195359,megawatthours
6582,2022-01-01,US48,United States Lower 48,WAT,Hydro,Eastern,Eastern,808045,megawatthours


In [103]:
# it's the same code as the code above but for a single API query (i.e. without splitting API into several smaller queries in order to address 5000-row limit)

import pandas as pd
import requests # to request/download data via API
from datetime import datetime, timedelta # to break up start & end dates into smaller monthly intervals
from dateutil import rrule # to break up start & end dates into smaller monthly intervals
from dateutil.relativedelta import relativedelta # to break up start & end dates into smaller monthly intervals
from dotenv import load_dotenv # I need this to hide API key
import os # I need this to hide API key

# I need this to hide API key
load_dotenv()
api_key = os.getenv('api_key')

# https://api.eia.gov/v2/electricity/rto/daily-fuel-type-data/data/?frequency=daily&data[0]=value&facets[respondent][]=US48&facets[timezone][]=Eastern&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000
# Daily net generation by balancing authority and energy source. Source: Form EIA-930 Product: Hourly Electric Grid Monitor

# because each API request is capped at 5000 rows, I need to create a code to loop through start & end dates in 1-month increments & concatenatte the resuld to my DataFrame. See the code in the cell just below this one.
start_date = datetime(2019, 1, 1)
end_date = datetime(2022, 1, 1)

# Download raw data using API
URL = 'https://api.eia.gov/v2/electricity/rto/daily-fuel-type-data/data/?api_key=' + api_key + '&frequency=daily&data[0]=value&facets[respondent][]=US48&facets[timezone][]=Eastern&start=' + start_date.strftime('%Y-%m-%d') + '&end=' + end_date.strftime('%Y-%m-%d') + '&sort[0][column]=period&sort[0][direction]=desc&offset=0&out=json' # convert datetime format (e.g. '2019, 1, 1') to string format (e.g. '2019-01-01') for start and end dates so that these can be understood by this API request
#URL = 'https://api.eia.gov/v2/electricity/rto/daily-region-data/data/?api_key=' + api_key + '&frequency=daily&data[0]=value&facets[respondent][]=US48&facets[timezone][]=Eastern&facets[type][]=D&facets[type][]=NG&sort[0][column]=period&start=' + start_date.strftime('%Y-%m-%d') + '&end=' + end_date.strftime('%Y-%m-%d') + '&sort[0][direction]=desc&offset=0&out=json' # convert datetime format (e.g. '2019, 1, 1') to string format (e.g. '2019-01-01') for start and end dates so that these can be understood by this API request
#URL = 'https://api.eia.gov/series/?series_id=PET.MCRRIP12.M&api_key=' + api_key + '&out=json'
#URL = 'http://api.eia.gov/category/?api_key=' + api_key + '&category_id=3&out=json'
#URL = 'https://api.eia.gov/v2/electricity/retail-sales&api_key=' + api_key + '&out=json'
headers = {'Accept-Encoding': 'identity'}
r = requests.get(URL, headers=headers)
#print(r.text[:10000])
# df = pd.DataFrame(r.json())
json_data = r.json()

# convert json data format to pandas DataFrame, then extract data as 'list_data' list of dictionaries
df_json = pd.DataFrame(json_data)
list_data = df_json.at['data', 'response']

# convert data from list of dictionaries format to pandas DataFrame format
col_names = list_data[0].keys() # create list of column names as 'col_names' dictionary
col_names = list(col_names) # convert 'col_names' dictionary to list
df = pd.DataFrame(list_data, columns=col_names) # convert 'list_data' list of dictionaries to 'df' pandas DataFrame
df

,period,respondent,respondent-name,fueltype,type-name,timezone,timezone-description,value,value-units
0,2022-01-01,US48,United States Lower 48,COL,Coal,Eastern,Eastern,1924618,megawatthours
1,2022-01-01,US48,United States Lower 48,WND,Wind,Eastern,Eastern,1485777,megawatthours
2,2022-01-01,US48,United States Lower 48,WAT,Hydro,Eastern,Eastern,808045,megawatthours
3,2022-01-01,US48,United States Lower 48,SUN,Solar,Eastern,Eastern,195359,megawatthours
4,2022-01-01,US48,United States Lower 48,OTH,Other,Eastern,Eastern,184933,megawatthours
...,...,...,...,...,...,...,...,...,...
4995,2020-04-17,US48,United States Lower 48,SUN,Solar,Eastern,Eastern,228341,megawatthours
4996,2020-04-17,US48,United States Lower 48,OTH,Other,Eastern,Eastern,200564,megawatthours
4997,2020-04-17,US48,United States Lower 48,OIL,Petroleum,Eastern,Eastern,9092,megawatthours
4998,2020-04-17,US48,United States Lower 48,NUC,Nuclear,Eastern,Eastern,1991699,megawatthours
